In [2]:
import pandas as pd
import numpy as np
import math
import string
import copy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from tqdm import tqdm
import warnings
import os
warnings.filterwarnings("ignore")

In [42]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 25
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    
    embed_dim = 128  # Embedding size for each token
    vocab_size = n_word_unique + 1

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = layers.Embedding(input_dim = vocab_size, output_dim = embed_dim, input_length=maxlen)
    x = embedding_layer(inputs)
    x = layers.Bidirectional(layers.SimpleRNN(64, input_dim=(None, embed_dim), 
                                         return_sequences=False, activation='tanh', 
                                         dropout=0.5))(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "rnn_models/{itr:04}/".format(itr=cnt)
    checkpoint_path = model_path + "{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir $model_path
    
    
    model.compile(tf.keras.optimizers.Adam(lr=0.0039), "binary_crossentropy", metrics=["accuracy", 
                                                          tf.keras.metrics.AUC(name='auc')])
    
    
    #create checkpoint to save model
    #with best validation loss
    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_auc', verbose=1,
            save_weights_only=True, save_best_only=True, mode='max')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=32, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )

    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

mkdir: cannot create directory ‘rnn_models/0001/’: No such file or directory
Epoch 1/10
250/250 [==============================] - ETA: 0s - loss: 0.6525 - accuracy: 0.6034 - auc: 0.6501
Epoch 00001: val_auc improved from -inf to 0.69255, saving model to rnn_models/0001/0001.ckpt
250/250 [==============================] - 9s 37ms/step - loss: 0.6525 - accuracy: 0.6034 - auc: 0.6501 - val_loss: 0.6604 - val_accuracy: 0.6220 - val_auc: 0.6926
Epoch 2/10
250/250 [==============================] - ETA: 0s - loss: 0.2363 - accuracy: 0.9081 - auc: 0.9654
Epoch 00002: val_auc improved from 0.69255 to 0.86517, saving model to rnn_models/0001/0002.ckpt
250/250 [==============================] - 9s 34ms/step - loss: 0.2363 - accuracy: 0.9081 - auc: 0.9654 - val_loss: 0.4799 - val_accuracy: 0.8050 - val_auc: 0.8652
Epoch 3/10
250/250 [==============================] - ETA: 0s - loss: 0.0491 - accuracy: 0.9839 - auc: 0.9977
Epoch 00003: val_auc improved from 0.86517 to 0.87334, saving model to rnn

In [43]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))
print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))

print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))


test avg loss:  0.5945095658302307 +/- 0.11129135729074856
test avg acc:  0.8358000040054321 +/- 0.011973302414260216
test avg auc:  0.8786239624023438 +/- 0.007943058672086837


val avg loss:  0.5494118392467499 +/- 0.1055582455506885
val avg acc:  0.8439999938011169 +/- 0.007127405409789778
val avg auc:  0.888472831249237 +/- 0.00934997085439449


train avg loss:  0.020650030672550203 +/- 0.015338121264560503
train avg acc:  0.9935750007629395 +/- 0.005653861806435143
train avg auc:  0.9993881344795227 +/- 0.0003174726321213222
